In [ ]:
import os
import sys
import sys
# Path追加
sys.path.append('../')
import numpy as np
from mmdet.apis import init_detector, inference_detector

import cv2
import imageio
import matplotlib.pyplot as plt

from common.visulaizations import draw_bb
from data import Dataset


DATA_DIR = "../data/Problem_01"
RESULT_DIR = "../results/Problem_01"

dataset = Dataset(DATA_DIR, RESULT_DIR, fps=5)

# Detection (By Takenobu 2022/09/16)

In [ ]:
# detection部分を関数化
def detection(img):
    config_path = '../configs/faster_rcnn_r50_fpn.py'
    weight_path = '../weights/latest.pth'
    

    model = init_detector(config_path, weight_path, device ='cuda:1')
    
    result_dir = '../results/nobu_result'

    score_th = 0.5

    #img = imageio.imread(img)
    result = inference_detector(model, img)
    
    return result

    '''
    det_img = draw_bb(img, result, score_th)
    det_img2 = cv2.cvtColor(det_img, cv2.COLOR_BGR2RGB)
    #plt.imshow(det_img2)
    cv2.imwrite(result_dir+os.path.basename(img_path), det_img)
    '''

# Try to make Object Tracking

In [ ]:
import math
from turtle import color
import cv2

# path to vidio
video_path = '../data/Problem_01/Problem_01.MP4'
cap = cv2.VideoCapture(video_path)
# Initialize Counter
count = 0
center_points_prev_frame = []

tracking_objects = {}
track_id = 0


while True:

    # Prepare for frame
    ret, frame = cap.read()

    # Point Current frame
    center_points_cur_frame = []    
    
    # If video is finished, escape from loop
    if not ret:
        break

    # Detection
    result = detection(frame)

    # Calculate the Center of detected fishes
    # each of loop is category of fish
    for category in result:
        # each of loop is data of detected fish
        for detected in category:
            x, y, x_w, y_h, score = detected[0:5]

            # Calculate center
            cx = int((x+x_w)/2)
            cy = int((y+y_h)/2)
            center_points_cur_frame.append((cx,cy))

            #cv2.circle(frame, (cx, cy), 10 , (0,0,255),-1)

    # Tracking
    # Only at the begining we compare previous and current freame
    if count <= 2:      
        for pt in center_points_cur_frame:
            for pt2 in center_points_prev_frame:
                # Calculate the distance between two points as Euclidean Distance
                distance = math.hypot(pt2[0]-pt[0], pt2[1]-pt2[1])
                # If distance is lower than 10, regard it as a same point
                if distance < 10:
                    tracking_objects[track_id] = pt
                    track_id += 1
    else:
        tracking_objects_copy = tracking_objects.copy()
        center_points_cur_frame_copy = center_points_cur_frame.copy()
           
        for object_id, pt2 in tracking_objects_copy.items():
            object_exits = False
            for pt in center_points_cur_frame:            
                distance = math.hypot(pt2[0]-pt[0], pt2[1]-pt2[1])
                # Update IDs position
                if distance < 30:
                    tracking_objects[object_id] = pt
                    object_exits = True
                    if pt in center_points_cur_frame:
                        center_points_cur_frame.remove(pt)
                        continue
            # Remove IDs lost
            if not object_exits:
                tracking_objects.pop(object_id)
    
        # Add new IDs found 
        for pt in center_points_cur_frame:
            tracking_objects[track_id] = pt
            track_id += 1


  
    for object_id, pt in  tracking_objects.items():
        cv2.circle(frame, pt, 5, (0,0, 255), -1)
        cv2.putText(frame, text=str(object_id), org=(pt[0], pt[1]-7), color=(0,0, 255),thickness=2,fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1.0)

    print("Tracking Object")
    print(tracking_objects)

    print("CUR FRAME LEFT PT")
    print(center_points_cur_frame)
    
    # Make a copy of the points
    center_points_prev_frame = center_points_cur_frame.copy()

    count += 1

    # Debug
    
    frame2 = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    plt.imshow(frame2)
    cv2.imwrite("../results/nobu_result/tracking_frame{}.png".format(count),frame)
    

    '''
    ## control the number of frames
    if count == 2:
        break
    '''

    # print(x,y,x_w,y_h,score)


cap.release()


## Video of Result 

In [ ]:
import cv2

img_dir = "../results/nobu_result"
# encoder(for mp4)
fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
# output file name, encoder, fps, size(fit to image size)
video = cv2.VideoWriter('../results/tracking_video_test1.mp4',fourcc, 30.0, (1920, 1080))

for i in range(1,421):
    img_path = img_dir + "/tracking_frame{}.png".format(i)
    img = cv2.imread(img_path)
    print(img)
    video.write(img)

video.release()
print('written')